# Satellite Data Extraction and Feature Engineering

This notebook extracts spectral band values from satellite imagery for specific forest plots and calculates various vegetation indices. The extracted features will be used for Above Ground Biomass (AGB) modeling.

### 1. Import Dependencies

In [1]:
import os
import pandas as pd
import numpy as np
import rasterio
from rasterio.sample import sample_gen
from pyproj import Transformer
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import glob

### 2. Configuration and Paths

In [2]:
BASE_DIR = "../../data"
FIELD_DATA_PATH = os.path.join(BASE_DIR, "field-data/final_target.csv")
SATELLITE_DIR = os.path.join(BASE_DIR, "satellite-images/2014")
OUTPUT_PATH = os.path.join(BASE_DIR, "satellite-data/final_data.csv")

# Bands of interest (Landsat 8)
BANDS = ["SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B6", "SR_B7"]

### 3. Load Field Data

In [3]:
df = pd.read_csv(FIELD_DATA_PATH)
print(f"Loaded {len(df)} plots from {FIELD_DATA_PATH}")
df.head()

Loaded 524 plots from ../../data/field-data/final_target.csv


,plot_lon,plot_lat,plot_agb_mean_t_ha,plot_total_agb_t,subplot_count,plotId
0,80.414795,28.870564,477.230220,71.584533,2.0,10-75
1,80.405569,29.192646,367.626079,27.571956,1.0,10-84
2,80.394871,29.483126,45.633926,10.267633,3.0,10-92
3,84.104420,28.228486,135.266012,40.579803,4.0,100-56
4,84.145391,27.650959,329.968995,148.486048,6.0,101-40


### 4. Pixel Extraction Logic

For each plot, we will locate the corresponding band images and extract the pixel value at the plot's coordinate (latitude and longitude). 

**Note:** Since satellite images are in UTM and our coordinates are in WGS84 (lat/lon), we must transform the coordinates to the image's CRS before sampling.

In [4]:
def extract_pixel_values(df, satellite_dir, bands):
    """
    Extracts pixel values for each band at the given lon/lat coordinates.
    """
    results = []
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Extracting Band Values"):
        plot_id = row['plotId']
        lon = row['plot_lon']
        lat = row['plot_lat']
        
        plot_features = {'plotId': plot_id}
        plot_folder = os.path.join(satellite_dir, f"plot-{plot_id}")
        
        if not os.path.exists(plot_folder):
            # print(f"Warning: No folder found for plot {plot_id}")
            for band in bands:
                plot_features[band] = np.nan
        else:
            for band in bands:
                band_file = os.path.join(plot_folder, f"plot-{plot_id}_{band}.tif")
                if os.path.exists(band_file):
                    try:
                        with rasterio.open(band_file) as src:
                            # Transform (lon, lat) to image CRS
                            transformer = Transformer.from_crs("EPSG:4326", src.crs, always_xy=True)
                            x, y = transformer.transform(lon, lat)
                            
                            coords = [(x, y)]
                            sampled_values = list(src.sample(coords))
                            plot_features[band] = float(sampled_values[0][0])
                    except Exception as e:
                        print(f"Error reading {band} for plot {plot_id}: {e}")
                        plot_features[band] = np.nan
                else:
                    plot_features[band] = np.nan
        
        results.append(plot_features)
    
    return pd.DataFrame(results)

### 5. Execute Extraction

In [5]:
band_features_df = extract_pixel_values(df, SATELLITE_DIR, BANDS)

Extracting Band Values: 100%|██████████| 524/524 [00:07<00:00, 65.56it/s] 


### 6. Merge and Calculate Vegetation Indices

We will merge the extracted band values back to our main dataframe and calculate standard vegetation indices like NDVI, EVI, and SAVI.

In [6]:
# Merge band features
final_df = df.merge(band_features_df, on='plotId', how='left')

# Mapping Landsat 8 bands to names for readability in formulas
# Blue: B2, Green: B3, Red: B4, NIR: B5, SWIR1: B6, SWIR2: B7
B2 = final_df['SR_B2']
B3 = final_df['SR_B3']
B4 = final_df['SR_B4']
B5 = final_df['SR_B5']
B6 = final_df['SR_B6']
B7 = final_df['SR_B7']

# 1. NDVI (Normalized Difference Vegetation Index)
final_df['NDVI'] = (B5 - B4) / (B5 + B4)

# 2. EVI (Enhanced Vegetation Index)
# Constants for EVI: G=2.5, C1=6, C2=7.5, L=1
final_df['EVI'] = 2.5 * ((B5 - B4) / (B5 + 6 * B4 - 7.5 * B2 + 1))

# 3. SAVI (Soil Adjusted Vegetation Index)
# L factor usually 0.5
L = 0.5
final_df['SAVI'] = ((B5 - B4) / (B5 + B4 + L)) * (1 + L)

# 4. NDWI (Normalized Difference Water Index) - Gao (1996)
final_df['NDWI'] = (B5 - B6) / (B5 + B6)

print("Feature extraction and index calculation complete.")
final_df.head()

Feature extraction and index calculation complete.


,plot_lon,plot_lat,plot_agb_mean_t_ha,plot_total_agb_t,subplot_count,plotId,SR_B2,SR_B3,SR_B4,SR_B5,SR_B6,SR_B7,NDVI,EVI,SAVI,NDWI
0,80.414795,28.870564,477.230220,71.584533,2.0,10-75,7671.0,8318.0,7894.0,15015.0,10514.0,8629.0,0.310839,3.672512,0.466248,0.176309
1,80.405569,29.192646,367.626079,27.571956,1.0,10-84,7380.0,7741.0,7599.0,10929.0,8535.0,7809.0,0.179728,7.091141,0.269585,0.122996
2,80.394871,29.483126,45.633926,10.267633,3.0,10-92,7724.0,8763.0,9095.0,15600.0,14208.0,11456.0,0.263414,1.328527,0.395112,0.046699
3,84.104420,28.228486,135.266012,40.579803,4.0,100-56,7855.0,8903.0,8507.0,15591.0,12162.0,9632.0,0.293966,2.293596,0.440940,0.123554
4,84.145391,27.650959,329.968995,148.486048,6.0,101-40,8222.0,9129.0,8227.0,12197.0,10767.0,9057.0,0.194379,-94.523810,0.291562,0.062271


### 7. Handle Missing Values and Save

Some plots might not have satellite data (due to clouds or being outside the range). We should decide how to handle these before saving.

In [7]:
missing_counts = final_df[BANDS + ['NDVI']].isnull().sum()
print("Missing values per feature:")
print(missing_counts)

# Drop rows where satellite data is missing if necessary, or keep them for imputation later
# For now, we save it as is.
final_df.to_csv(OUTPUT_PATH, index=False)
print(f"Final dataset saved to {OUTPUT_PATH}")

Missing values per feature:
SR_B2    0
SR_B3    0
SR_B4    0
SR_B5    0
SR_B6    0
SR_B7    0
NDVI     1
dtype: int64
Final dataset saved to ../../data/satellite-data/final_data.csv
